# Digit Classification with Neural Networks 

Interest in neural networks, and in particular those with architechures that support deep learning, has been surging in recent years.  

In this notebook we will be revisiting the problem of digit classification on the MNIST data.  In doing so, we will introduce a new Python library, Theano, for working with neural networks.  Theano is a popular choice for neural networks as the same code can be run on either CPUs or GPUs.  GPUs greatly speed up the training and prediction, and is readily available. Amazon offers GPU machines on EC2.  

In part 1, we'll introduce Theano, and refresh ourselves on the MNIST dataset.  In part 2, we'll create a multi-layer neural network with a simple architechure, and train it using backpropagation.  Part 3 will introduce the convolutional architechure, which can be said to be doing 'deep learning' (also called feature learning or representation learning).

#### Part 1: Basics

Lets start to look at Theano.  If later you'd like to go deeper into Theano, you may want to read this paper: http://www.iro.umontreal.ca/~lisa/pointeurs/theano_scipy2010.pdf

Install Theano if you haven't already.  Then let's load it, and set it to work with a CPU.  For reference, here is the Theano documentation: http://www.deeplearning.net/software/theano/library/

In [17]:
%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import time

import warnings
warnings.filterwarnings('ignore')

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print(theano.config.device) # We're using CPUs (for now)
print(theano.config.floatX) # Should be 64 bit for CPUs

np.random.seed(0)

cpu
float64


Now for MNIST data...

In [2]:
# Repeating steps from Project 1 to prepare mnist dataset. 
X, Y = fetch_openml(name='mnist_784', return_X_y=True, cache=False)
X = X / 255.0

shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]
numExamples = 2000
test_data, test_labels = X[70000-numExamples:], Y[70000-numExamples:]
train_data, train_labels = X[:numExamples], Y[:numExamples]
numFeatures = train_data[1].size
numTrainExamples = train_data.shape[0]
numTestExamples = test_data.shape[0]
print('Features = ', numFeatures)
print('Train set = ', numTrainExamples)
print('Test set = ', numTestExamples)

Features =  784
Train set =  2000
Test set =  2000


Looking ahead to working with neural networks, let's prepare one additional variation of the label data.  Let's make these labels, rather than each being an integer value from 0-9, be a set of 10 binary values, one for each class.  This is sometimes called a 1-of-n encoding, and it makes working with Neural Networks easier, as there will be one output node for each class.

In [3]:
def binarizeY(data):
    binarized_data = np.zeros((data.size,10))
    for j in range(0,data.size):
        feature = data[j:j+1]
        i = feature.astype(np.int64) 
        binarized_data[j,i]=1
    return binarized_data
train_labels_b = binarizeY(train_labels)
test_labels_b = binarizeY(test_labels)
numClasses = train_labels_b[1].size
print('Classes = ', numClasses)

Classes =  10


Lets start with a KNN model to establish a baseline accuracy.

Exercise: You've seen a number of different classification algorithms (e.g. naive bayes, decision trees, random forests, logistic regression) at this point.  How does KNN scalability and accuracy with respect to the size of the training dataset compare to those other algorithms?  

In [4]:
neighbors = 1
knn = KNeighborsClassifier(neighbors)
# we'll be waiting quite a while if we use 60K examples, so let's cut it down.  You may want to run the full 60K on your own later to see what the accuracy is.
mini_train_data, mini_train_labels = X[:numExamples], Y[:numExamples] 
start_time = time.time()
knn.fit(mini_train_data, mini_train_labels)
print('Train time = ', time.time() - start_time)
start_time = time.time()
accuracy = knn.score(test_data, test_labels)
print('Accuracy = ', accuracy)
print('Prediction time = ', time.time() - start_time)

Train time =  0.03586220741271973
Accuracy =  0.9065
Prediction time =  4.524647951126099


Alright, now that we have a simple baseline, let's start working in Theano.  Before we jump to multi-layer neural networks though, let's train a logistic regression model to make certain we're using Theano correctly. 

Recall from Josh's regression lecture the four key components: (1) parameters, (2) model, (3) cost function, and (4) objective. 

In [5]:
## (1) Parameters 
# Initialize the weights to small, but non-zero, values.
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

Two notes relevant at this point:

First, logistic regression can be thought of as a neural network with no hidden layers. The output values are just the dot product of the inputs and the edge weights.

Second, we have 10 classes. We can either train separate one vs all classifiers using sigmoid activation, which would be a hassle, or we can use the softmax activation, which is essentially a multi-class version of sigmoid. We'll use Theano's built-in implementation of softmax.

In [6]:
## (2) Model
# Theano objects accessed with standard Python variables
X = T.matrix()
Y = T.matrix()

def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

We'll use cross-entropy as a cost function.  Cross entropy only considers the error between the true class and the prediction, and not the errors for the false classes.  This tends to cause the network to converge faster.  We'll use Theano's built-in cross entropy function.

In [7]:
## (3) Cost function
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

The objective is minimize the cost, and to do that we'll use batch gradient descent.

Exercise: What are the differences between batch, stochastic, and mini-batch gradient descent?  What are the implications of each for working on large datasets?

We'll use Theano's built-in gradient function.  Exercise: Do you recall from Josh's lecture what the gradient is for beta in logistic regression?

In [8]:
## (4) Objective (and solver)

alpha = 0.01
gradient = T.grad(cost=cost, wrt=w) 
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) # computes cost, then runs update
y_pred = T.argmax(y_hat, axis=1) # select largest probability as prediction
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

def gradientDescent(epochs):
    trainTime = 0.0
    predictTime = 0.0
    for i in range(epochs):
        start_time = time.time()
        cost = train(train_data[0:len(train_data)], train_labels_b[0:len(train_data)])
        trainTime =  trainTime + (time.time() - start_time)
        print('accuracy = ', (i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = ', trainTime)

gradientDescent(50)

start_time = time.time()
predict(test_data)   
print('predict time = ', (time.time() - start_time))

accuracy =  (1, 0.097)
accuracy =  (2, 0.1245)
accuracy =  (3, 0.1555)
accuracy =  (4, 0.183)
accuracy =  (5, 0.2195)
accuracy =  (6, 0.249)
accuracy =  (7, 0.283)
accuracy =  (8, 0.3075)
accuracy =  (9, 0.342)
accuracy =  (10, 0.3645)
accuracy =  (11, 0.3905)
accuracy =  (12, 0.4105)
accuracy =  (13, 0.4265)
accuracy =  (14, 0.447)
accuracy =  (15, 0.4635)
accuracy =  (16, 0.4765)
accuracy =  (17, 0.488)
accuracy =  (18, 0.501)
accuracy =  (19, 0.513)
accuracy =  (20, 0.5235)
accuracy =  (21, 0.535)
accuracy =  (22, 0.543)
accuracy =  (23, 0.552)
accuracy =  (24, 0.5575)
accuracy =  (25, 0.5645)
accuracy =  (26, 0.5695)
accuracy =  (27, 0.5745)
accuracy =  (28, 0.5855)
accuracy =  (29, 0.5895)
accuracy =  (30, 0.5985)
accuracy =  (31, 0.6045)
accuracy =  (32, 0.609)
accuracy =  (33, 0.612)
accuracy =  (34, 0.6155)
accuracy =  (35, 0.6195)
accuracy =  (36, 0.624)
accuracy =  (37, 0.626)
accuracy =  (38, 0.6285)
accuracy =  (39, 0.631)
accuracy =  (40, 0.6325)
accuracy =  (41, 0.635)
ac

Exercise:  What do you expect to happen if we convert batch gradient descent to stochastic gradient descent?  Why?

Let's try it...

In [9]:
## (1) Parameters
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('accuracy = ', (i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = ', trainTime)
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = ', (time.time() - start_time))

accuracy =  (1, 0.841)
accuracy =  (2, 0.8665)
accuracy =  (3, 0.872)
accuracy =  (4, 0.8755)
accuracy =  (5, 0.8775)
accuracy =  (6, 0.8775)
accuracy =  (7, 0.878)
accuracy =  (8, 0.8795)
accuracy =  (9, 0.88)
accuracy =  (10, 0.879)
accuracy =  (11, 0.88)
accuracy =  (12, 0.8815)
accuracy =  (13, 0.881)
accuracy =  (14, 0.8795)
accuracy =  (15, 0.8795)
accuracy =  (16, 0.879)
accuracy =  (17, 0.8795)
accuracy =  (18, 0.8795)
accuracy =  (19, 0.879)
accuracy =  (20, 0.879)
accuracy =  (21, 0.8795)
accuracy =  (22, 0.879)
accuracy =  (23, 0.879)
accuracy =  (24, 0.8785)
accuracy =  (25, 0.88)
accuracy =  (26, 0.8795)
accuracy =  (27, 0.8795)
accuracy =  (28, 0.88)
accuracy =  (29, 0.88)
accuracy =  (30, 0.8805)
accuracy =  (31, 0.881)
accuracy =  (32, 0.8805)
accuracy =  (33, 0.88)
accuracy =  (34, 0.8795)
accuracy =  (35, 0.8785)
accuracy =  (36, 0.8785)
accuracy =  (37, 0.8795)
accuracy =  (38, 0.879)
accuracy =  (39, 0.879)
accuracy =  (40, 0.879)
accuracy =  (41, 0.8795)
accuracy =

Exercise: What do you expect to happen if you switch the batch size to be great than 1 (i.e. mini-batch)?  Why?

Try it for a few values...

In [10]:
## (1) Parameters
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 10 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('accuracy = ', (i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))     
    print('train time = ', trainTime)
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = ', (time.time() - start_time))

accuracy =  (1, 0.788)
accuracy =  (2, 0.8245)
accuracy =  (3, 0.8395)
accuracy =  (4, 0.851)
accuracy =  (5, 0.8565)
accuracy =  (6, 0.861)
accuracy =  (7, 0.865)
accuracy =  (8, 0.867)
accuracy =  (9, 0.8685)
accuracy =  (10, 0.869)
accuracy =  (11, 0.873)
accuracy =  (12, 0.875)
accuracy =  (13, 0.8755)
accuracy =  (14, 0.876)
accuracy =  (15, 0.8775)
accuracy =  (16, 0.8765)
accuracy =  (17, 0.8785)
accuracy =  (18, 0.88)
accuracy =  (19, 0.88)
accuracy =  (20, 0.88)
accuracy =  (21, 0.8805)
accuracy =  (22, 0.881)
accuracy =  (23, 0.8805)
accuracy =  (24, 0.881)
accuracy =  (25, 0.882)
accuracy =  (26, 0.8825)
accuracy =  (27, 0.8825)
accuracy =  (28, 0.8835)
accuracy =  (29, 0.8835)
accuracy =  (30, 0.883)
accuracy =  (31, 0.8825)
accuracy =  (32, 0.883)
accuracy =  (33, 0.883)
accuracy =  (34, 0.883)
accuracy =  (35, 0.8825)
accuracy =  (36, 0.882)
accuracy =  (37, 0.882)
accuracy =  (38, 0.882)
accuracy =  (39, 0.883)
accuracy =  (40, 0.8835)
accuracy =  (41, 0.8835)
accuracy =

#### PART 2: Multi-layer Neural Networks

As we start to get further into Neural Networks, if you'd like to take time on your own for an in-depth introduction on the state of the art in the topic, check out this excellent 1-day tutorial from KDD2014:

Part 1: http://videolectures.net/kdd2014_bengio_deep_learning/

Part 2: http://videolectures.net/tcmm2014_taylor_deep_learning/

---------

Let's take our implementation of logistic regression (which recall is in fact a single layer neural network), and add a hidden layer, making it a two layer neural network.  Because we have a hidden layer, we will now train the model using backpropagation.

Exercise: How do you expect this model to compare to KNN and logistic regression in terms of train time and accuracy?  Why?

Let's try it out...

In [11]:
## (1) Parameters
numHiddenNodes = 600 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2]


## (2) Model
X = T.matrix()
Y = T.matrix()
# Two notes:
# First, feed forward is the composition of layers (dot product + activation function)
# Second, activation on the hidden layer still uses sigmoid
def model(X, w_1, w_2):
    return T.nnet.softmax(T.dot(T.nnet.sigmoid(T.dot(X, w_1)), w_2))
y_hat = model(X, w_1, w_2)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.3500
2) accuracy = 0.6240
3) accuracy = 0.8025
4) accuracy = 0.8415
5) accuracy = 0.8545
6) accuracy = 0.8650
7) accuracy = 0.8700
8) accuracy = 0.8745
9) accuracy = 0.8810
10) accuracy = 0.8825
11) accuracy = 0.8855
12) accuracy = 0.8875
13) accuracy = 0.8910
14) accuracy = 0.8885
15) accuracy = 0.8865
16) accuracy = 0.8855
17) accuracy = 0.8870
18) accuracy = 0.8870
19) accuracy = 0.8860
20) accuracy = 0.8845
21) accuracy = 0.8840
22) accuracy = 0.8850
23) accuracy = 0.8845
24) accuracy = 0.8840
25) accuracy = 0.8830
26) accuracy = 0.8825
27) accuracy = 0.8820
28) accuracy = 0.8820
29) accuracy = 0.8815
30) accuracy = 0.8810
31) accuracy = 0.8800
32) accuracy = 0.8800
33) accuracy = 0.8805
34) accuracy = 0.8810
35) accuracy = 0.8805
36) accuracy = 0.8800
37) accuracy = 0.8795
38) accuracy = 0.8790
39) accuracy = 0.8780
40) accuracy = 0.8785
41) accuracy = 0.8785
42) accuracy = 0.8785
43) accuracy = 0.8790
44) accuracy = 0.8785
45) accuracy = 0.8780
46) accuracy = 0.87

Exercise: Change the number of nodes in the hidden layer?  What do you expect the impact to be?  What is the impact?

--------

As interest in networks with more layers and more complicated architechures has increased, a couple of tricks have emerged and become standard practice.  Let's look at two of those--rectifier activation and dropout noise.

Exercise:  We saw an improvement from adding a hidden layer.  What do you expect to happen if a second hidden layer was added?  

Let's try it...

In [12]:
## (1) Parms
numHiddenNodes = 600 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numHiddenNodes))*.01)))
w_3 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2, w_3]

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w_1, w_2, w_3):
    return T.nnet.softmax(T.dot(T.nnet.sigmoid(T.dot(T.nnet.sigmoid(T.dot(X, w_1)), w_2)), w_3))
y_hat = model(X, w_1, w_2, w_3)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(10)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.1105
2) accuracy = 0.1105
3) accuracy = 0.1105
4) accuracy = 0.0965
5) accuracy = 0.0965
6) accuracy = 0.0965
7) accuracy = 0.1135
8) accuracy = 0.1135
9) accuracy = 0.1730
10) accuracy = 0.2480
train time = 66.98
predict time = 0.04


#### Activation Revisted

Let's look at a recent idea around activation closely associated with deep learning.  In 2010, in a paper published at NIPS (https://www.utc.fr/~bordesan/dokuwiki/_media/en/glorot10nipsworkshop.pdf), Yoshua Bengio showed that rectifier activation works better empirically than sigmoid activation when used in the hidden layers.  

The rectifier activation is simple: f(x)=max(0,x).  Intuitively, the difference is that as a sigmoid activated node approaches 1 it stops learning even if error continues to be propagated to it, whereas the rectifier activated node continue to learn (at least in the positive direction).  It is not completely understood (per Yoshua Bengio) why this helps, but there are some theories being explored including as related to the benefits of sparse representations in networks. (http://www.iro.umontreal.ca/~bengioy/talks/KDD2014-tutorial.pdf).  Rectifiers also speed up training.

Although the paper was published in 2010, the technique didn't gain widespread adoption until 2012 when members of Hinton's group spread the word, including with this Kaggle entry: http://blog.kaggle.com/2012/11/01/deep-learning-how-i-did-it-merck-1st-place-interview/

Let's change the activation in our 2 layer network to rectifier and see what happens...

In [13]:
## (1) Parms
numHiddenNodes = 600 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2]


## (2) Model
X = T.matrix()
Y = T.matrix()

def model(X, w_1, w_2):
    return T.nnet.softmax(T.dot(T.maximum(T.dot(X, w_1), 0.), w_2))
y_hat = model(X, w_1, w_2)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.8260
2) accuracy = 0.8665
3) accuracy = 0.8760
4) accuracy = 0.8915
5) accuracy = 0.8945
6) accuracy = 0.8975
7) accuracy = 0.9015
8) accuracy = 0.9020
9) accuracy = 0.9030
10) accuracy = 0.9040
11) accuracy = 0.9065
12) accuracy = 0.9070
13) accuracy = 0.9075
14) accuracy = 0.9065
15) accuracy = 0.9075
16) accuracy = 0.9080
17) accuracy = 0.9080
18) accuracy = 0.9105
19) accuracy = 0.9105
20) accuracy = 0.9110
21) accuracy = 0.9120
22) accuracy = 0.9115
23) accuracy = 0.9120
24) accuracy = 0.9115
25) accuracy = 0.9120
26) accuracy = 0.9120
27) accuracy = 0.9120
28) accuracy = 0.9110
29) accuracy = 0.9115
30) accuracy = 0.9120
31) accuracy = 0.9125
32) accuracy = 0.9125
33) accuracy = 0.9130
34) accuracy = 0.9130
35) accuracy = 0.9130
36) accuracy = 0.9130
37) accuracy = 0.9125
38) accuracy = 0.9120
39) accuracy = 0.9120
40) accuracy = 0.9120
41) accuracy = 0.9115
42) accuracy = 0.9120
43) accuracy = 0.9125
44) accuracy = 0.9125
45) accuracy = 0.9135
46) accuracy = 0.91

#### Maxout Activation

So rectifier activation worked great!  

Exercise: try another type of activation called Maxout (or Max Pooling) activiation.  Maxout activation just selects the max input as the output.  Maxout is a type of pooling, a technique which performs particularly well for vision problems. For more background see: http://jmlr.org/proceedings/papers/v28/goodfellow13.pdf and http://www.quora.com/What-is-impact-of-different-pooling-methods-in-convolutional-neural-networks  

#### Noise

Previously when working with the MNIST data we saw a benefit in generalization from adding noise to the training data.  Let's try that again here, however this time with a trick for adding noise called 'Dropouts'.  The idea with dropouts is that instead of (or in addition to) adding noise to our inputs, we add noise by having each node return 0 with a certain probability during training.  This trick both improves generalization in large networks and speeds up training.

Hinton introduced the idea in 2012 and gave an explanation of why it's similar to bagging (http://arxiv.org/pdf/1207.0580v1.pdf)

Let's give it a try...

In [14]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

## (1) Parms
numHiddenNodes = 600 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2]


## (2) Model
X = T.matrix()
Y = T.matrix()
srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

def model(X, w_1, w_2, p_1, p_2):
    return T.nnet.softmax(T.dot(dropout(T.maximum(T.dot(dropout(X, p_1), w_1),0.), p_2), w_2))

y_hat_train = model(X, w_1, w_2, 0.2, 0.5)
y_hat_predict = model(X, w_1, w_2, 0., 0.)

## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat_train, Y))

## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat_predict, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)


miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.7735
2) accuracy = 0.8600
3) accuracy = 0.8620
4) accuracy = 0.8810
5) accuracy = 0.8960
6) accuracy = 0.9020
7) accuracy = 0.8925
8) accuracy = 0.8980
9) accuracy = 0.9185
10) accuracy = 0.9175
11) accuracy = 0.9180
12) accuracy = 0.9160
13) accuracy = 0.9125
14) accuracy = 0.9215
15) accuracy = 0.9265
16) accuracy = 0.9245
17) accuracy = 0.9275
18) accuracy = 0.9215
19) accuracy = 0.9280
20) accuracy = 0.9300
21) accuracy = 0.9280
22) accuracy = 0.9205
23) accuracy = 0.9260
24) accuracy = 0.9235
25) accuracy = 0.9350
26) accuracy = 0.9315
27) accuracy = 0.9295
28) accuracy = 0.9340
29) accuracy = 0.9280
30) accuracy = 0.9295
31) accuracy = 0.9275
32) accuracy = 0.9340
33) accuracy = 0.9315
34) accuracy = 0.9275
35) accuracy = 0.9315
36) accuracy = 0.9295
37) accuracy = 0.9315
38) accuracy = 0.9305
39) accuracy = 0.9310
40) accuracy = 0.9305
41) accuracy = 0.9310
42) accuracy = 0.9290
43) accuracy = 0.9270
44) accuracy = 0.9305
45) accuracy = 0.9310
46) accuracy = 0.92

Let's try once again adding a second hidden layer...

In [15]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

## (1) Parmeters
numHiddenNodes = 600 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numHiddenNodes))*.01)))
w_3 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2, w_3]


## (2) Model
X = T.matrix()
Y = T.matrix()

srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

def model(X, w_1, w_2, w_3, p_1, p_2, p_3):
    return T.nnet.softmax(T.dot(dropout(T.maximum(T.dot(dropout(T.maximum(T.dot(dropout(X, p_1), w_1),0.), p_2), w_2),0.), p_3), w_3))

y_hat_train = model(X, w_1, w_2, w_3, 0.2, 0.5,0.5)
y_hat_predict = model(X, w_1, w_2, w_3, 0., 0.,0.)

## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat_train, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat_predict, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.5410
2) accuracy = 0.7920
3) accuracy = 0.8130
4) accuracy = 0.8545
5) accuracy = 0.8570
6) accuracy = 0.8765
7) accuracy = 0.8880
8) accuracy = 0.9040
9) accuracy = 0.8875
10) accuracy = 0.9160
11) accuracy = 0.9000
12) accuracy = 0.9130
13) accuracy = 0.9170
14) accuracy = 0.9290
15) accuracy = 0.9085
16) accuracy = 0.9255
17) accuracy = 0.9200
18) accuracy = 0.9260
19) accuracy = 0.9175
20) accuracy = 0.9250
21) accuracy = 0.9255
22) accuracy = 0.9215
23) accuracy = 0.9315
24) accuracy = 0.9180
25) accuracy = 0.9270
26) accuracy = 0.9295
27) accuracy = 0.9280
28) accuracy = 0.9220
29) accuracy = 0.9290
30) accuracy = 0.9230
31) accuracy = 0.9260
32) accuracy = 0.9215
33) accuracy = 0.9285
34) accuracy = 0.9285
35) accuracy = 0.9145
36) accuracy = 0.9020
37) accuracy = 0.9315
38) accuracy = 0.9320
39) accuracy = 0.9255
40) accuracy = 0.9205
41) accuracy = 0.9290
42) accuracy = 0.9280
43) accuracy = 0.9260
44) accuracy = 0.9250
45) accuracy = 0.9315
46) accuracy = 0.93

#### Part 3: Convolutional Neural Nets

Today, when the phrase 'deep learning' is used to describe a system, it is often a convolution neural network (or convonet).  The convonet architechture was largely developed in the late 90's at Bell Labs, but only very recently popularized.  It was developed for image recognition, and is described and implemented with 2d representations in mind.

Geoffrey Hinton has an excellent two-part lecture on the topic:

https://www.youtube.com/watch?v=6oD3t6u5EPs

https://www.youtube.com/watch?v=fueIAeAsGzA

Also, this code was partly taken from these tutorials, which are worth referring back to:

http://deeplearning.net/tutorial/lenet.html

http://ufldl.stanford.edu/tutorial/supervised/FeatureExtractionUsingConvolution/

https://www.youtube.com/watch?v=S75EdAcXHKk

http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/

In [18]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.pool import pool_2d

## (1) Parameters
numHiddenNodes = 600 
patchWidth = 3
patchHeight = 3
featureMapsLayer1 = 32
featureMapsLayer2 = 64
featureMapsLayer3 = 128

# For convonets, we will work in 2d rather than 1d.  The MNIST images are 28x28 in 2d.
imageWidth = 28
train_data = train_data.reshape(-1, 1, imageWidth, imageWidth)
test_data = test_data.reshape(-1, 1, imageWidth, imageWidth)

# Convolution layers.  
w_1 = theano.shared(np.asarray((np.random.randn(*(featureMapsLayer1, 1, patchWidth, patchHeight))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(featureMapsLayer2, featureMapsLayer1, patchWidth, patchHeight))*.01)))
w_3 = theano.shared(np.asarray((np.random.randn(*(featureMapsLayer3, featureMapsLayer2, patchWidth, patchHeight))*.01)))

# Fully connected NN. 
w_4 = theano.shared(np.asarray((np.random.randn(*(featureMapsLayer3 * 3 * 3, numHiddenNodes))*.01)))
w_5 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2, w_3, w_4, w_5]

## (2) Model
X = T.tensor4() # conv2d works with tensor4 type
Y = T.matrix()

srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

# Theano provides built-in support for add convolutional layers
def model(X, w_1, w_2, w_3, w_4, w_5, p_1, p_2):
    l1 = dropout(pool_2d(T.maximum(conv2d(X, w_1, border_mode='full'),0.), (2, 2)), p_1)
    l2 = dropout(pool_2d(T.maximum(conv2d(l1, w_2), 0.), (2, 2)), p_1)
    l3 = dropout(T.flatten(pool_2d(T.maximum(conv2d(l2, w_3), 0.), (2, 2)), outdim=2), p_1) # flatten to switch back to 1d layers
    l4 = dropout(T.maximum(T.dot(l3, w_4), 0.), p_2)
    return T.nnet.softmax(T.dot(l4, w_5))

y_hat_train = model(X, w_1, w_2, w_3, w_4, w_5, 0.2, 0.5)
y_hat_predict = model(X, w_1, w_2, w_3, w_4, w_5, 0., 0.)
y_x = T.argmax(y_hat, axis=1)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat_train, Y))

## (4) Minimization.  
def backprop(cost, w, alpha=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        
        # adding gradient scaling
        acc = theano.shared(w1.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * grad ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        grad = grad / gradient_scaling
        updates.append((acc, acc_new))
        
        updates.append((w1, w1 - grad * alpha))
    return updates

update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat_predict, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(10)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.8315
2) accuracy = 0.9140
3) accuracy = 0.9380
4) accuracy = 0.9495
5) accuracy = 0.9550
6) accuracy = 0.9540
7) accuracy = 0.9500
8) accuracy = 0.9675
9) accuracy = 0.9605
10) accuracy = 0.9585
train time = 3113.23
predict time = 7.04


#### Additional topic: Brain inspiration

The architechture of the convonet was inspired by the visual cortext in the human brain.  If you are interested in learning more, check out: http://www-psych.stanford.edu/~ashas/Cognition%20Textbook/chapter2.pdf

#### Additional topic: Self-Driving Vehicles

Convolutional networks are starting to be used more and more for self-driving vehicles.  This past year, NVIDEA started produced a chipset for self-driving vehicles that analyzes the video of up to 14 cameras using convonets for 

CES 2015 parts 6,7,9

https://www.youtube.com/watch?v=-vKGkxeflGw

https://www.youtube.com/watch?v=zsVsUvx8ieo

https://www.youtube.com/watch?v=RvQVyGOynFY

GTC 2015 parts 4,5,7,9

https://www.youtube.com/watch?v=pqvdZ2jp1NA

https://www.youtube.com/watch?v=GGxdP_JWhwI

https://www.youtube.com/watch?v=Tb7ZYSTYHbw

https://www.youtube.com/watch?v=TDm6Snkle70